In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Extract data from pdf files
def load_pdf_file(data):
    loader =DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf_file(data='Data/')

Traceback (most recent call last):
  File "c:\Users\HP\.vscode\extensions\ms-python.python-2025.0.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "<string>", line 6, in load_pdf_file
  File "d:\ML_Projects\Medical_Chatbot_Gen_AI\medibot\Lib\site-packages\langchain_community\document_loaders\directory.py", line 117, in load
    return list(self.lazy_load())
           ^^^^^^^^^^^^^^^^^^^^^^
  File "d:\ML_Projects\Medical_Chatbot_Gen_AI\medibot\Lib\site-packages\langchain_community\document_loaders\directory.py", line 123, in lazy_load
    raise FileNotFoundError(f"Directory not found: '{self.path}'")
FileNotFoundError: Directory not found: 'Data/'

